In [1]:
import psycopg2
import pandas as pd
import geopandas as gpd
import unicodedata
import warnings
from shapely import wkb

# Ignorar advertencias
warnings.filterwarnings('ignore')

# ==========================================
# 0. FUNCIONES
# ==========================================

def safe_read_parquet(path):
    """Lee archivos parquet con manejo robusto."""
    try:
        return pd.read_parquet(path, engine='pyarrow')
    except Exception as e:
        print(f"Error PyArrow: {e}. Intentando fastparquet...")
        return pd.read_parquet(path, engine='fastparquet')

def normalize_text(text):
    if isinstance(text, str):
        text = text.strip().upper()
        return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    return text

# ==========================================
# 1. CARGA DE PUNTOS BLUE (SQL)
# ==========================================

print("--- Paso 1: Descargando Puntos Blue (SQL) ---")

DB_HOST = "dwh.datarq.blue.internal"
DB_NAME = "dwh"
DB_USER = "molivares" 
DB_PASS = "MA2012"

try:
    con = psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST, port="5432")
    cur = con.cursor()
    sql = "SELECT cmns_nmb, geol_latitud, geol_longitud, estado FROM reports.pickup_maestro_v02 WHERE estado = 'active';"
    cur.execute(sql)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    con.close()
    
    PUDOS_Bx = pd.DataFrame(rows, columns=columns)
    
    # Limpieza coordenadas
    PUDOS_Bx['geol_latitud'] = PUDOS_Bx['geol_latitud'].astype(str).str.replace(',', '.').astype(float)
    PUDOS_Bx['geol_longitud'] = PUDOS_Bx['geol_longitud'].astype(str).str.replace(',', '.').astype(float)
    
    # Crear GeoDataFrame PUDO
    gdf_PUDO = gpd.GeoDataFrame(PUDOS_Bx, geometry=gpd.points_from_xy(PUDOS_Bx.geol_longitud, PUDOS_Bx.geol_latitud), crs="EPSG:4326")
    gdf_PUDO['COMUNA_NORM'] = gdf_PUDO['cmns_nmb'].apply(normalize_text)
    
    print(f"Puntos cargados: {len(gdf_PUDO)}")

except Exception as e:
    print(f"Error SQL: {e}")
    gdf_PUDO = gpd.GeoDataFrame(columns=['cmns_nmb', 'geometry'], crs="EPSG:4326")

# ==========================================
# 2. CARGA DE CENSO (MANZANAS)
# ==========================================

print("--- Paso 2: Cargando Censo 2024 (Manzanas) ---")

path_manzanas = r"C:\Users\martin.olivares\Downloads\Cartografia_censo2024_Pais\Cartografia_censo2024_Pais_Manzanas.parquet" 
df_manzanas = safe_read_parquet(path_manzanas)

# Geometría
col_geometria = 'geometry'
if 'SHAPE' in df_manzanas.columns:
    col_geometria = 'SHAPE'

if df_manzanas[col_geometria].dtype == 'object' or isinstance(df_manzanas[col_geometria].iloc[0], bytes):
    print("Convirtiendo WKB...")
    df_manzanas[col_geometria] = df_manzanas[col_geometria].apply(lambda x: wkb.loads(x) if isinstance(x, bytes) else x)

manzanas = gpd.GeoDataFrame(df_manzanas, geometry=col_geometria)
if manzanas.crs is None:
    manzanas.set_crs(epsg=4326, inplace=True)

# Población
if 'n_per' in manzanas.columns:
    manzanas.rename(columns={'n_per': 'TOTAL_PERS'}, inplace=True)
elif 'TOTAL_PERS' not in manzanas.columns:
    manzanas['TOTAL_PERS'] = 0

# ==========================================
# 3. PREPARACIÓN DE DATOS (NIVEL MANZANA)
# ==========================================

print("--- Paso 3: Preparando Datos (Nivel Manzana) ---")

manzanas_full = manzanas.copy()

# 1. Asegurar ID Único de Manzana (MANZENT)
if 'MANZENT' in manzanas_full.columns:
    # Convertir a string para evitar notación científica y asegurar unicidad exacta
    manzanas_full['MANZENT'] = manzanas_full['MANZENT'].fillna(0).astype('int64').astype(str)
else:
    print("ADVERTENCIA: No se encontró MANZENT. Creando ID sintético.")
    manzanas_full['MANZENT'] = manzanas_full.index.astype(str)

# 2. Rellenar Nulos en Jerarquía
cols_fill = {
    'ID_ENTIDAD': 0,
    'ENTIDAD': 'DISPERSO / RURAL',
    'CATEGORIA': 'DISPERSO',
    'LOCALIDAD': 'DESCONOCIDA',
    'AREA_C': 'RURAL/OTRO',
    'COMUNA': 'SIN COMUNA'
}

for col, val in cols_fill.items():
    if col not in manzanas_full.columns:
        manzanas_full[col] = val
    else:
        if col == 'ID_ENTIDAD':
             manzanas_full[col] = manzanas_full[col].fillna(0).astype('int64')
        else:
             manzanas_full[col] = manzanas_full[col].fillna(val)

# Ajuste específico para ID_ENTIDAD=0 -> Rural
mask_rural = manzanas_full['ID_ENTIDAD'] == 0
manzanas_full.loc[mask_rural, 'ENTIDAD'] = 'DISPERSO / RURAL'
manzanas_full.loc[mask_rural, 'CATEGORIA'] = 'DISPERSO'

# Normalizar Comuna
manzanas_full['COMUNA_NORM'] = manzanas_full['COMUNA'].apply(normalize_text)

# Proyección UTM
print("Proyectando a UTM 19S...")
manzanas_utm = manzanas_full.to_crs(epsg=32719)
gdf_pudo_utm = gdf_PUDO.to_crs(epsg=32719)

# ==========================================
# 4. CÁLCULO DE COBERTURA POR MANZANA
# ==========================================

print("--- Paso 4: Calculando Cobertura por Manzana ---")

resultados_manzanas = []
grupos_comuna = manzanas_utm.groupby('COMUNA_NORM')
total = len(grupos_comuna)
count = 0

for comuna_norm, df_mz_comuna in grupos_comuna:
    count += 1
    if count % 20 == 0:
        print(f"Procesando Comuna {count}/{total}: {comuna_norm}")

    # Filtrar PUDOs
    pudos_comuna = gdf_pudo_utm[gdf_pudo_utm['COMUNA_NORM'] == comuna_norm]
    
    # Preparar datos base de la comuna
    df_mz_comuna = df_mz_comuna.copy()
    df_mz_comuna['area_total_m2'] = df_mz_comuna.geometry.area
    
    # Columnas a guardar
    cols_output = [
        'MANZENT', 'COMUNA', 'ENTIDAD', 'CATEGORIA', 'LOCALIDAD', 'AREA_C', 
        'TOTAL_PERS', 'area_total_m2', 'ID_ENTIDAD', 'ID_LOCALIDAD'
    ]
    # Filtrar solo las que existen
    cols_validas = [c for c in cols_output if c in df_mz_comuna.columns]
    
    # Indexar por MANZENT para mapeo rápido
    base_data = df_mz_comuna[cols_validas].set_index('MANZENT')
    
    # Inicializar métricas
    base_data['area_cubierta_m2'] = 0.0
    base_data['pudos_en_comuna'] = len(pudos_comuna)
    base_data['pudos_en_manzana'] = 0 # Nueva columna

    if not pudos_comuna.empty:
        try:
            # --- A. CÁLCULO DE COBERTURA (ÁREA) ---
            # 1. Buffer Union (800m)
            buffer_union = pudos_comuna.geometry.buffer(800).unary_union
            gdf_buffer = gpd.GeoDataFrame(geometry=[buffer_union], crs=df_mz_comuna.crs)
            
            # 2. Intersección Espacial
            # overlay devuelve las geometrías resultantes de la intersección
            interseccion = gpd.overlay(df_mz_comuna, gdf_buffer, how='intersection')
            
            if not interseccion.empty:
                # 3. Calcular área intersectada
                interseccion['area_pedazo'] = interseccion.geometry.area
                
                # 4. Sumar áreas por MANZENT (una manzana puede tener múltiples pedazos intersectados)
                areas_cubiertas = interseccion.groupby('MANZENT')['area_pedazo'].sum()
                
                # 5. Asignar al dataframe base
                base_data.loc[areas_cubiertas.index, 'area_cubierta_m2'] = areas_cubiertas
            
            # --- B. CONTEO DE PUDOS CERCANOS (Por Manzana) ---
            # Creamos buffers individuales para contar cuántos tocan cada manzana
            pudos_buffers = pudos_comuna.copy()
            pudos_buffers['geometry'] = pudos_buffers.geometry.buffer(800)
            
            # Spatial Join: Manzanas vs Buffers Individuales
            # Usamos sjoin para ver qué manzana toca qué buffer
            # Detectar nombre de columna de geometría
            geom_col_name = df_mz_comuna.geometry.name
            mz_for_sjoin = df_mz_comuna[['MANZENT', geom_col_name]].copy()
            mz_for_sjoin = gpd.GeoDataFrame(mz_for_sjoin, geometry=geom_col_name, crs=df_mz_comuna.crs)
            
            joined_pudos = gpd.sjoin(mz_for_sjoin, pudos_buffers[['geometry']], how='inner', predicate='intersects')
            
            # Contar ocurrencias por MANZENT
            counts_pudos = joined_pudos.groupby('MANZENT').size()
            base_data.loc[counts_pudos.index, 'pudos_en_manzana'] = counts_pudos

        except Exception as e:
            print(f"Error en comuna {comuna_norm}: {e}")

    # Calcular Porcentajes y Población
    base_data['pct_cobertura'] = (base_data['area_cubierta_m2'] / base_data['area_total_m2']).fillna(0.0)
    base_data['pct_cobertura'] = base_data['pct_cobertura'].clip(upper=1.0) # Corregir errores flotantes > 1.0
    
    base_data['pob_cubierta'] = base_data['TOTAL_PERS'] * base_data['pct_cobertura']
    
    # Agregar a resultados
    resultados_manzanas.append(base_data.reset_index())

# ==========================================
# 5. CONSOLIDACIÓN Y GUARDADO
# ==========================================

print("--- Paso 5: Guardando Reporte Granular ---")

if resultados_manzanas:
    df_final = pd.concat(resultados_manzanas, ignore_index=True)
    
    # Redondeo
    df_final['pob_cubierta'] = df_final['pob_cubierta'].round(2)
    df_final['pct_cobertura'] = df_final['pct_cobertura'].round(4)
    df_final['area_cubierta_m2'] = df_final['area_cubierta_m2'].round(1)
    df_final['area_total_m2'] = df_final['area_total_m2'].round(1)
    
    # Asegurar tipo entero para conteos
    df_final['pudos_en_manzana'] = df_final['pudos_en_manzana'].astype(int)
    df_final['pudos_en_comuna'] = df_final['pudos_en_comuna'].astype(int)

    # Ordenar
    df_final = df_final.sort_values(by=['COMUNA', 'ENTIDAD', 'MANZENT'])
    
    filename = 'Cobertura_Censo24_Por_Manzana.xlsx'
    
    # Guardar (puede ser pesado, usar engine eficiente si es necesario)
    print(f"Generando Excel con {len(df_final)} filas...")
    df_final.to_excel(filename, index=False)
    print(f"Archivo generado: {filename}")
else:
    print("No se generaron resultados.")

--- Paso 1: Descargando Puntos Blue (SQL) ---
Puntos cargados: 3275
--- Paso 2: Cargando Censo 2024 (Manzanas) ---
Convirtiendo WKB...
--- Paso 3: Preparando Datos (Nivel Manzana) ---
Proyectando a UTM 19S...
--- Paso 4: Calculando Cobertura por Manzana ---
Procesando Comuna 20/334: CALBUCO
Procesando Comuna 40/334: CHIGUAYANTE
Procesando Comuna 60/334: CONCEPCION
Procesando Comuna 80/334: DONIHUE
Procesando Comuna 100/334: HIJUELAS
Procesando Comuna 120/334: LA LIGUA
Procesando Comuna 140/334: LO ESPEJO
Procesando Comuna 160/334: MARCHIHUE
Procesando Comuna 180/334: NOGALES
Procesando Comuna 200/334: PEDRO AGUIRRE CERDA
Procesando Comuna 220/334: PORVENIR
Procesando Comuna 240/334: QUELLON
Procesando Comuna 260/334: REQUINOA
Procesando Comuna 280/334: SAN IGNACIO
Procesando Comuna 300/334: SANTO DOMINGO
Procesando Comuna 320/334: VALLENAR
--- Paso 5: Guardando Reporte Granular ---
Generando Excel con 216341 filas...
Archivo generado: Cobertura_Censo24_Por_Manzana.xlsx


In [3]:
# ==========================================
# 6. GENERACIÓN DE REPORTE AGRUPADO (ALDEA/LOCALIDAD)
# ==========================================

print("--- Paso 6: Generando Reporte Agrupado (Aldea/Localidad) ---")

# Usamos el DataFrame final de manzanas (df_final) si existe, o procesamos desde cero si es necesario.
# Asumimos que 'df_final' está en memoria del paso anterior. Si no, habría que cargarlo.

if 'df_final' in locals() and not df_final.empty:
    print("Agrupando datos desde el nivel Manzana...")
    
    # Definir columnas de agrupación
    cols_group = ['COMUNA', 'ID_ENTIDAD', 'ENTIDAD', 'CATEGORIA', 'LOCALIDAD', 'AREA_C']
    
    # Agregación básica
    df_agrupado = df_final.groupby(cols_group).agg({
        'TOTAL_PERS': 'sum',
        'pob_cubierta': 'sum',
        'area_total_m2': 'sum',
        'area_cubierta_m2': 'sum',
        'pudos_en_comuna': 'first' # Es el mismo para toda la comuna
    }).reset_index()
    
    # Recalcular porcentaje de cobertura poblacional
    df_agrupado['PORCENTAJE_COBERTURA'] = (df_agrupado['pob_cubierta'] / df_agrupado['TOTAL_PERS']).fillna(0)
    
    # --- CÁLCULO DE PUDOS POR ENTIDAD ---
    # Para contar cuántos PUDOs caen en la entidad, necesitamos la geometría de la entidad.
    # Como no tenemos la geometría de la entidad directa, la reconstruimos disolviendo las manzanas.
    
    print("Calculando PUDOs por Entidad (Spatial Join)...")
    
    # 1. Recuperar geometría de manzanas (necesitamos volver a unir con el GeoDataFrame original o usar lo que tenemos)
    # Es más eficiente filtrar los PUDOs por comuna y hacer el conteo espacial.
    
    pudos_por_entidad = []
    
    # Iterar por comuna para optimizar espacialmente
    for comuna in df_agrupado['COMUNA'].unique():
        # Filtrar datos de la comuna
        mask_comuna = df_agrupado['COMUNA'] == comuna
        entidades_comuna = df_agrupado[mask_comuna]
        
        # Obtener PUDOs de esta comuna (usando el gdf_pudo_utm global)
        # Normalizamos nombre para coincidir
        comuna_norm = normalize_text(comuna)
        pudos_locales = gdf_pudo_utm[gdf_pudo_utm['COMUNA_NORM'] == comuna_norm]
        
        if pudos_locales.empty:
            df_agrupado.loc[mask_comuna, 'CANTIDAD_PUDOS_ENTIDAD'] = 0
            continue
            
        # Para cada entidad en la comuna
        for idx, row in entidades_comuna.iterrows():
            id_entidad = row['ID_ENTIDAD']
            
            # Obtener las geometrías de las manzanas de esta entidad
            # Filtramos del GeoDataFrame 'manzanas_utm' original
            if id_entidad != 0:
                mz_entidad = manzanas_utm[manzanas_utm['ID_ENTIDAD'] == id_entidad]
            else:
                # Caso Disperso/Rural: Coincidir por nombre de entidad/localidad si ID es 0
                mz_entidad = manzanas_utm[
                    (manzanas_utm['COMUNA_NORM'] == comuna_norm) & 
                    (manzanas_utm['ID_ENTIDAD'] == 0) &
                    (manzanas_utm['ENTIDAD'] == row['ENTIDAD'])
                ]
            
            if mz_entidad.empty:
                pudos_count = 0
            else:
                # Crear geometría unificada de la entidad (dissolve)
                geom_entidad = mz_entidad.unary_union
                
                # Contar PUDOs que intersectan con la entidad (con un pequeño buffer de tolerancia si se desea, o directo)
                # Aquí contamos PUDOs que están DENTRO o MUY CERCA (ej. 800m buffer de la entidad)
                # OJO: El usuario pidió "pudos en la aldea". Usualmente es intersección con el área urbana.
                # Pero dado que usamos buffers de 800m para cobertura, quizás quiera saber cuántos PUDOs sirven a la aldea.
                # Opción A: PUDOs geográficamente DENTRO de la mancha urbana.
                # Opción B: PUDOs cuyo buffer de 800m toca la mancha urbana.
                
                # Usaremos Opción B (Alcance) para ser consistentes con la cobertura, 
                # PERO si dice "en la aldea", suele ser ubicación física.
                # Vamos a usar intersección del buffer del PUDO con la geometría de la entidad.
                
                # Buffer de PUDOs
                pudos_buffers = pudos_locales.geometry.buffer(800)
                
                # Verificar intersección
                intersects = pudos_buffers.intersects(geom_entidad)
                pudos_count = intersects.sum()
            
            df_agrupado.at[idx, 'CANTIDAD_PUDOS_ENTIDAD'] = pudos_count

    # Limpieza y formato
    df_agrupado['CANTIDAD_PUDOS_ENTIDAD'] = df_agrupado['CANTIDAD_PUDOS_ENTIDAD'].fillna(0).astype(int)
    df_agrupado['POBLACION_TOTAL'] = df_agrupado['TOTAL_PERS'].astype(int)
    df_agrupado['POBLACION_CUBIERTA'] = df_agrupado['pob_cubierta'].round(0).astype(int)
    df_agrupado['PORCENTAJE_COBERTURA'] = df_agrupado['PORCENTAJE_COBERTURA'].round(4)
    
    # Seleccionar columnas finales
    cols_final = [
        'COMUNA', 'ENTIDAD', 'CATEGORIA', 'LOCALIDAD', 'AREA_C', 'ID_ENTIDAD',
        'POBLACION_TOTAL', 'POBLACION_CUBIERTA', 'PORCENTAJE_COBERTURA',
        'CANTIDAD_PUDOS_ENTIDAD', 'pudos_en_comuna'
    ]
    
    df_resumen = df_agrupado[cols_final].sort_values(by=['COMUNA', 'LOCALIDAD', 'ENTIDAD'])
    
    # Guardar
    filename_resumen = 'Cobertura_Censo24_Resumen_Aldeas.xlsx'
    df_resumen.to_excel(filename_resumen, index=False)
    print(f"Archivo de resumen generado: {filename_resumen}")
    
    # Mostrar muestra
    print("\nEjemplo de resultados agrupados:")
    print(df_resumen)

else:
    print("Error: No se encontró el DataFrame 'df_final'. Ejecuta el paso anterior primero.")

--- Paso 6: Generando Reporte Agrupado (Aldea/Localidad) ---
Agrupando datos desde el nivel Manzana...
Calculando PUDOs por Entidad (Spatial Join)...
Archivo de resumen generado: Cobertura_Censo24_Resumen_Aldeas.xlsx

Ejemplo de resultados agrupados:
         COMUNA                ENTIDAD CATEGORIA  \
2     ALGARROBO              ALGARROBO    Ciudad   
0     ALGARROBO                EL YECO     Aldea   
1     ALGARROBO                MIRASOL    Pueblo   
4         ALHUÉ  IGNACIO CARRERA PINTO    Pueblo   
3         ALHUÉ         HACIENDA ALHUÉ     Aldea   
...         ...                    ...       ...   
1316   ZAPALLAR              CATAPILCO    Pueblo   
1317   ZAPALLAR  LA LAGUNA DE ZAPALLAR    Pueblo   
1318   ZAPALLAR    ZAPALLAR - CACHAGUA    Pueblo   
1319     ÑIQUÉN           SAN GREGORIO    Pueblo   
1320      ÑUÑOA                  ÑUÑOA    Ciudad   

                                              LOCALIDAD  AREA_C  \
2                       ALGARROBO - EL QUISCO - EL TABO  

In [4]:
# ==========================================
# 7. ANÁLISIS DE OPORTUNIDADES (DONDE MOVER LA AGUJA)
# ==========================================

print("--- Paso 7: Análisis de Oportunidades de Expansión ---")

# Contexto: Buscamos maximizar el impacto. 
# "Mover la aguja" significa capturar la mayor cantidad de población nueva con el menor esfuerzo (nuevos puntos).
# Estrategia: Identificar Entidades (Barrios/Aldeas/Ciudades) con alta población NO cubierta.

if 'df_resumen' in locals() and not df_resumen.empty:
    
    # 1. Calcular el GAP (Población Sin Cubrir)
    df_oportunidades = df_resumen.copy()
    df_oportunidades['POBLACION_SIN_CUBRIR'] = df_oportunidades['POBLACION_TOTAL'] - df_oportunidades['POBLACION_CUBIERTA']
    
    # 2. Clasificar el tipo de oportunidad
    def clasificar_oportunidad(row):
        if row['CANTIDAD_PUDOS_ENTIDAD'] == 0:
            return "EXPANSIÓN (Zona Nueva)"
        elif row['PORCENTAJE_COBERTURA'] < 0.5:
            return "DENSIFICACIÓN CRÍTICA (Cobertura < 50%)"
        elif row['PORCENTAJE_COBERTURA'] < 0.8:
            return "DENSIFICACIÓN MEDIA (Mejorar Servicio)"
        else:
            return "MANTENIMIENTO (Cobertura Alta)"

    df_oportunidades['TIPO_OPORTUNIDAD'] = df_oportunidades.apply(clasificar_oportunidad, axis=1)
    
    # 3. Ranking de Impacto
    # Ordenamos por la cantidad absoluta de personas que hoy NO atendemos
    df_ranking = df_oportunidades.sort_values(by='POBLACION_SIN_CUBRIR', ascending=False)
    
    # Filtramos casos triviales (ej. donde falta muy poca gente)
    df_ranking = df_ranking[df_ranking['POBLACION_SIN_CUBRIR'] > 100]

    # 4. Generar Reporte Estratégico
    cols_estrategia = [
        'COMUNA', 'ENTIDAD', 'CATEGORIA', 'LOCALIDAD', 
        'POBLACION_TOTAL', 'POBLACION_CUBIERTA', 'POBLACION_SIN_CUBRIR', 
        'PORCENTAJE_COBERTURA', 'CANTIDAD_PUDOS_ENTIDAD', 'TIPO_OPORTUNIDAD'
    ]
    
    df_estrategia = df_ranking[cols_estrategia]
    
    # Guardar
    filename_opp = 'Oportunidades_Expansion_PUDOs.xlsx'
    df_estrategia.to_excel(filename_opp, index=False)
    print(f"Reporte de Oportunidades generado: {filename_opp}")
    
    # --- VISUALIZACIÓN DE INSIGHTS ---
    print("\n=== TOP 10 LUGARES PARA 'MOVER LA AGUJA' (Mayor Población Sin Atender) ===")
    print(df_estrategia.head(10).to_string(index=False))
    
    print("\n=== TOP 5 OPORTUNIDADES 'GREENFIELD' (Lugares con 0 PUDOs y mucha gente) ===")
    greenfield = df_estrategia[df_estrategia['CANTIDAD_PUDOS_ENTIDAD'] == 0]
    print(greenfield.head(5).to_string(index=False))

else:
    print("Error: No se encontró 'df_resumen'. Ejecuta el paso 6 primero.")

--- Paso 7: Análisis de Oportunidades de Expansión ---
Reporte de Oportunidades generado: Oportunidades_Expansion_PUDOs.xlsx

=== TOP 10 LUGARES PARA 'MOVER LA AGUJA' (Mayor Población Sin Atender) ===
       COMUNA       ENTIDAD CATEGORIA                                LOCALIDAD  POBLACION_TOTAL  POBLACION_CUBIERTA  POBLACION_SIN_CUBRIR  PORCENTAJE_COBERTURA  CANTIDAD_PUDOS_ENTIDAD                        TIPO_OPORTUNIDAD
  ANTOFAGASTA   ANTOFAGASTA    Ciudad                              ANTOFAGASTA           387247              249930                137317                0.6454                      39  DENSIFICACIÓN MEDIA (Mejorar Servicio)
 VIÑA DEL MAR  VIÑA DEL MAR    Ciudad                          GRAN VALPARAÍSO           330109              202404                127705                0.6131                      57  DENSIFICACIÓN MEDIA (Mejorar Servicio)
 SAN BERNARDO  SAN BERNARDO    Ciudad                            GRAN SANTIAGO           290334              184743            

In [6]:
# ==========================================
# 8. MAPA INTERACTIVO DE COBERTURA NACIONAL
# ==========================================

print("--- Paso 8: Generando Mapa Interactivo Nacional ---")

import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap
from branca.colormap import LinearColormap
import numpy as np

# ==========================================
# A. PREPARACIÓN DE DATOS PARA EL MAPA
# ==========================================

print("Preparando datos geoespaciales...")

# 1. Convertir manzanas a EPSG:4326 (lat/lon) para Folium
manzanas_wgs84 = manzanas_utm.to_crs(epsg=4326)

# 2. Calcular centroides de cada manzana
manzanas_wgs84['centroid'] = manzanas_wgs84.geometry.centroid
manzanas_wgs84['lat'] = manzanas_wgs84['centroid'].y
manzanas_wgs84['lon'] = manzanas_wgs84['centroid'].x

# 3. Unir con métricas calculadas (df_final tiene pct_cobertura)
# Usar MANZENT como key
df_map = manzanas_wgs84[['MANZENT', 'COMUNA', 'ENTIDAD', 'LOCALIDAD', 'TOTAL_PERS', 'lat', 'lon']].copy()
df_map = df_map.merge(
    df_final[['MANZENT', 'pct_cobertura', 'pob_cubierta', 'pudos_en_manzana']], 
    on='MANZENT', 
    how='left'
)
df_map['pct_cobertura'] = df_map['pct_cobertura'].fillna(0)
df_map['pob_cubierta'] = df_map['pob_cubierta'].fillna(0)
df_map['pudos_en_manzana'] = df_map['pudos_en_manzana'].fillna(0)

# 4. Calcular población sin cubrir
df_map['pob_sin_cubrir'] = df_map['TOTAL_PERS'] - df_map['pob_cubierta']
df_map['pob_sin_cubrir'] = df_map['pob_sin_cubrir'].clip(lower=0)

# 5. Filtrar manzanas con población > 0 para el heatmap
df_map_poblado = df_map[df_map['TOTAL_PERS'] > 0].copy()

print(f"Manzanas con población: {len(df_map_poblado):,}")

# ==========================================
# B. CREAR MAPA BASE
# ==========================================

print("Creando mapa base...")

# Centro de Chile (aproximado)
chile_center = [-33.45, -70.65]  # Santiago como centro inicial

mapa = folium.Map(
    location=chile_center,
    zoom_start=6,
    tiles=None,  # Agregaremos tiles personalizados
    control_scale=True
)

# Agregar múltiples capas de tiles
folium.TileLayer('cartodbpositron', name='Mapa Claro').add_to(mapa)
folium.TileLayer('cartodbdark_matter', name='Mapa Oscuro').add_to(mapa)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(mapa)

# ==========================================
# C. CAPA 1: HEATMAP DE POBLACIÓN SIN COBERTURA
# ==========================================

print("Generando HeatMap de población sin cobertura...")

# Preparar datos para heatmap: [lat, lon, peso]
# El peso es la población SIN cubrir (donde necesitamos expandir)
df_sin_cob = df_map_poblado[df_map_poblado['pob_sin_cubrir'] > 0].copy()

# =============================================
# FIX: Usar escala LOGARÍTMICA para diferenciar magnitudes
# =============================================
# Problema: Una manzana con 50 personas sin cubrir se ve igual que una con 5000
# Solución: log(población) para que las diferencias sean visibles
# 
# Ejemplo de escala:
#   - 10 personas  → log10(10) = 1.0   → peso ~0.25
#   - 100 personas → log10(100) = 2.0  → peso ~0.50
#   - 1000 personas → log10(1000) = 3.0 → peso ~0.75
#   - 10000 personas → log10(10000) = 4.0 → peso ~1.0

# Aplicar logaritmo base 10
df_sin_cob['log_pob'] = np.log10(df_sin_cob['pob_sin_cubrir'].clip(lower=1))

# Normalizar: máximo teórico ~4.5 (para ~30,000 personas por manzana)
log_max = 4.0  # log10(10000) - ajustar según tus datos
df_sin_cob['peso_norm'] = (df_sin_cob['log_pob'] / log_max).clip(upper=1.0)

# Filtrar puntos con peso muy bajo (ruido visual)
df_sin_cob = df_sin_cob[df_sin_cob['peso_norm'] > 0.1]  # Mínimo ~12 personas

heat_data_sin_cobertura = df_sin_cob[['lat', 'lon', 'peso_norm']].values.tolist()

print(f"   Puntos en heatmap: {len(heat_data_sin_cobertura):,}")
print(f"   Rango de pesos: {df_sin_cob['peso_norm'].min():.2f} - {df_sin_cob['peso_norm'].max():.2f}")

heatmap_sin_cobertura = HeatMap(
    heat_data_sin_cobertura,
    name='🔴 Población SIN Cobertura (Oportunidad)',
    min_opacity=0.2,
    max_zoom=18,
    radius=12,  # Reducido para mejor definición
    blur=8,
    gradient={
        0.0: 'transparent',
        0.15: 'lightyellow',  # Muy poca gente
        0.3: 'yellow',        # ~20-50 personas
        0.5: 'orange',        # ~100-300 personas
        0.7: 'red',           # ~500-1000 personas
        0.85: 'darkred',      # ~2000-5000 personas
        1.0: 'purple'         # ~10000+ personas
    }
)
heatmap_sin_cobertura.add_to(mapa)

# ==========================================
# D. CAPA 2: HEATMAP DE POBLACIÓN CON COBERTURA (VERDE)
# ==========================================

print("Generando HeatMap de población cubierta...")

df_cubierta = df_map_poblado[df_map_poblado['pob_cubierta'] > 0].copy()

# También usar escala logarítmica para consistencia
df_cubierta['log_pob'] = np.log10(df_cubierta['pob_cubierta'].clip(lower=1))
df_cubierta['peso_norm'] = (df_cubierta['log_pob'] / log_max).clip(upper=1.0)
df_cubierta = df_cubierta[df_cubierta['peso_norm'] > 0.1]

heat_data_cubierta = df_cubierta[['lat', 'lon', 'peso_norm']].values.tolist()

heatmap_cubierta = HeatMap(
    heat_data_cubierta,
    name='🟢 Población CON Cobertura',
    min_opacity=0.2,
    max_zoom=18,
    radius=12,
    blur=8,
    gradient={
        0.0: 'transparent',
        0.2: 'palegreen',
        0.4: 'lightgreen',
        0.6: 'green',
        0.8: 'darkgreen',
        1.0: 'darkslategray'
    },
    show=False  # Oculta por defecto
)
heatmap_cubierta.add_to(mapa)

# ==========================================
# E. CAPA 3: MARCADORES DE PUDOs (TIENDAS)
# ==========================================

print("Agregando marcadores de PUDOs...")

# Usar MarkerCluster para no saturar el mapa
pudo_cluster = MarkerCluster(
    name='📍 PUDOs Activos',
    show=True,
    options={
        'disableClusteringAtZoom': 14,
        'spiderfyOnMaxZoom': True
    }
)

# Convertir PUDOs a WGS84
gdf_pudo_wgs84 = gdf_PUDO.to_crs(epsg=4326)

for idx, row in gdf_pudo_wgs84.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"<b>PUDO</b><br>Comuna: {row['cmns_nmb']}",
        icon=folium.Icon(color='blue', icon='store', prefix='fa')
    ).add_to(pudo_cluster)

pudo_cluster.add_to(mapa)

# ==========================================
# F. CAPA 4: TOP OPORTUNIDADES (Círculos grandes)
# ==========================================

print("Agregando TOP oportunidades de expansión...")

# Agregar por Entidad (usando df_estrategia del paso anterior)
if 'df_estrategia' in locals() and not df_estrategia.empty:
    
    # Crear FeatureGroup para oportunidades
    fg_oportunidades = folium.FeatureGroup(name='⭐ TOP Oportunidades Expansión', show=True)
    
    # Obtener centroides de las top 50 entidades con más población sin cubrir
    top_50 = df_estrategia.head(50)
    
    for idx, row in top_50.iterrows():
        # Buscar centroide de la entidad
        if row['ENTIDAD'] != 'DISPERSO / RURAL':
            mask = (df_map['ENTIDAD'] == row['ENTIDAD']) & (df_map['COMUNA'] == row['COMUNA'])
            subset = df_map[mask]
            if not subset.empty:
                lat_mean = subset['lat'].mean()
                lon_mean = subset['lon'].mean()
                
                # Tamaño del círculo proporcional a población sin cubrir
                radius = min(max(row['POBLACION_SIN_CUBRIR'] / 100, 500), 5000)
                
                # Color según tipo de oportunidad
                if row['CANTIDAD_PUDOS_ENTIDAD'] == 0:
                    color = 'red'
                    tipo = '🚨 NUEVA ZONA'
                elif row['PORCENTAJE_COBERTURA'] < 0.5:
                    color = 'orange'
                    tipo = '⚠️ CRÍTICO'
                else:
                    color = 'yellow'
                    tipo = '📊 MEJORAR'
                
                popup_html = f"""
                <div style='width:200px'>
                    <h4>{row['ENTIDAD']}</h4>
                    <b>Comuna:</b> {row['COMUNA']}<br>
                    <b>Categoría:</b> {row['CATEGORIA']}<br>
                    <b>Población Total:</b> {row['POBLACION_TOTAL']:,}<br>
                    <b>Sin Cubrir:</b> {row['POBLACION_SIN_CUBRIR']:,}<br>
                    <b>Cobertura:</b> {row['PORCENTAJE_COBERTURA']*100:.1f}%<br>
                    <b>PUDOs actuales:</b> {row['CANTIDAD_PUDOS_ENTIDAD']}<br>
                    <hr>
                    <b style='color:{color}'>{tipo}</b>
                </div>
                """
                
                folium.CircleMarker(
                    location=[lat_mean, lon_mean],
                    radius=10,
                    color=color,
                    fill=True,
                    fillColor=color,
                    fillOpacity=0.7,
                    popup=folium.Popup(popup_html, max_width=250)
                ).add_to(fg_oportunidades)
    
    fg_oportunidades.add_to(mapa)

# ==========================================
# G. CAPA 5: ZONAS SOBRESERVIDAS (Para optimizar)
# ==========================================

print("Identificando zonas sobreservidas...")

# Manzanas con muchos PUDOs cercanos (> 3) y alta cobertura
df_sobreservido = df_map_poblado[
    (df_map_poblado['pudos_en_manzana'] > 3) & 
    (df_map_poblado['pct_cobertura'] > 0.9)
]

if len(df_sobreservido) > 0:
    fg_sobreservido = folium.FeatureGroup(name='🔵 Zonas Sobreservidas (Optimizar)', show=False)
    
    # Agrupar por comuna para no saturar
    for comuna in df_sobreservido['COMUNA'].unique()[:30]:  # Limitar a 30 comunas
        subset = df_sobreservido[df_sobreservido['COMUNA'] == comuna]
        if len(subset) > 5:  # Solo si hay suficientes manzanas
            lat_mean = subset['lat'].mean()
            lon_mean = subset['lon'].mean()
            n_pudos = subset['pudos_en_manzana'].sum()
            
            folium.CircleMarker(
                location=[lat_mean, lon_mean],
                radius=8,
                color='blue',
                fill=True,
                fillColor='blue',
                fillOpacity=0.5,
                popup=f"<b>{comuna}</b><br>Alta densidad de PUDOs<br>Considerar consolidación"
            ).add_to(fg_sobreservido)
    
    fg_sobreservido.add_to(mapa)

# ==========================================
# H. AGREGAR CONTROLES Y LEYENDA
# ==========================================

print("Agregando controles...")

# Mini mapa
MiniMap(toggle_display=True).add_to(mapa)

# Control de capas
folium.LayerControl(collapsed=False).add_to(mapa)

# Leyenda HTML personalizada
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 240px; 
    border:2px solid grey; z-index:9999; 
    background-color:white;
    padding: 10px;
    font-size: 12px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
<b style="font-size:14px">📊 Mapa de Cobertura Blue</b><br><br>
<b>HeatMap Sin Cobertura (Escala Log):</b><br>
<span style="color:lightyellow;background:#333;padding:0 3px">■</span> Amarillo claro = ~20-50 pers.<br>
<span style="color:yellow;background:#333;padding:0 3px">■</span> Amarillo = ~50-100 pers.<br>
<span style="color:orange">■</span> Naranja = ~100-500 pers.<br>
<span style="color:red">■</span> Rojo = ~500-2000 pers.<br>
<span style="color:darkred">■</span> Rojo oscuro = ~2000-5000 pers.<br>
<span style="color:purple">■</span> Púrpura = 5000+ pers.<br><br>
<b>Marcadores:</b><br>
<span style="color:blue">●</span> PUDOs actuales<br>
<span style="color:red">●</span> Zona nueva (0 PUDOs)<br>
<span style="color:orange">●</span> Cobertura crítica (<50%)<br>
</div>
'''
mapa.get_root().html.add_child(folium.Element(legend_html))

# ==========================================
# I. GUARDAR MAPA
# ==========================================

filename_mapa = 'Mapa_Cobertura_Nacional_Blue.html'
mapa.save(filename_mapa)

print(f"\n✅ Mapa interactivo generado: {filename_mapa}")
print(f"   Tamaño aproximado del archivo: {len(heat_data_sin_cobertura):,} puntos de calor")
print("\n📌 INSTRUCCIONES DE USO:")
print("   1. Abre el archivo HTML en tu navegador")
print("   2. Usa el panel de capas (arriba derecha) para activar/desactivar vistas")
print("   3. Zoom in/out para ver detalle por manzana")
print("   4. Click en marcadores para ver información detallada")
print("   5. HeatMap ROJO = Donde ir a buscar nuevos puntos")
print("   6. HeatMap VERDE = Donde ya tenemos cobertura")

--- Paso 8: Generando Mapa Interactivo Nacional ---
Preparando datos geoespaciales...
Manzanas con población: 166,497
Creando mapa base...
Generando HeatMap de población sin cobertura...
   Puntos en heatmap: 60,052
   Rango de pesos: 0.10 - 0.90
Generando HeatMap de población cubierta...
Agregando marcadores de PUDOs...
Agregando TOP oportunidades de expansión...
Identificando zonas sobreservidas...
Agregando controles...

✅ Mapa interactivo generado: Mapa_Cobertura_Nacional_Blue.html
   Tamaño aproximado del archivo: 60,052 puntos de calor

📌 INSTRUCCIONES DE USO:
   1. Abre el archivo HTML en tu navegador
   2. Usa el panel de capas (arriba derecha) para activar/desactivar vistas
   3. Zoom in/out para ver detalle por manzana
   4. Click en marcadores para ver información detallada
   5. HeatMap ROJO = Donde ir a buscar nuevos puntos
   6. HeatMap VERDE = Donde ya tenemos cobertura


In [8]:
# ==========================================
# 9. MAPA DE OPORTUNIDADES RURALES (HUB LOGÍSTICO)
# ==========================================
# Concepto: Identificar comunas rurales donde un solo PUDO puede ser el 
# "centro logístico" que atrae a toda la población dispersa de la comuna.
# Ejemplo: Quinchao (isla) - un PUDO sirve a toda la población rural que 
# viaja al pueblo principal para hacer sus envíos.

print("--- Paso 9: Análisis de Oportunidades Rurales (Hub Logístico) ---")

import folium
from folium.plugins import MarkerCluster, MiniMap
import numpy as np

# ==========================================
# A. EXPLORAR DATOS PARA ENTENDER CATEGORÍAS
# ==========================================

print("Explorando estructura de datos del censo...")
print(f"\nColumnas disponibles: {list(manzanas_full.columns)}")
print(f"\nValores únicos en CATEGORIA: {manzanas_full['CATEGORIA'].unique()}")
print(f"\nValores únicos en AREA_C: {manzanas_full['AREA_C'].unique()}")
print(f"\nValores únicos en ID_ENTIDAD (primeros 20): {sorted(manzanas_full['ID_ENTIDAD'].unique())[:20]}")

# Ver distribución de población por categoría
print("\n--- Población por CATEGORIA ---")
pop_by_cat = manzanas_full.groupby('CATEGORIA')['TOTAL_PERS'].sum().sort_values(ascending=False)
print(pop_by_cat)

print("\n--- Población por AREA_C ---")
pop_by_area = manzanas_full.groupby('AREA_C')['TOTAL_PERS'].sum().sort_values(ascending=False)
print(pop_by_area)

# ==========================================
# B. IDENTIFICAR COMUNAS CON ALTA POBLACIÓN RURAL
# ==========================================

print("\n\nAnalizando comunas con población rural dispersa...")

# 1. Agrupar manzanas por COMUNA y calcular métricas totales
df_comunas_rural = manzanas_full.groupby('COMUNA').agg({
    'TOTAL_PERS': 'sum',
    'MANZENT': 'count'
}).reset_index()
df_comunas_rural.columns = ['COMUNA', 'POBLACION_TOTAL', 'N_MANZANAS']

# 2. DEFINICIÓN DE RURAL - Basado en los datos reales
# Rural = ID_ENTIDAD == 0 (zonas sin entidad urbana definida)
# O áreas pequeñas que no son ciudades grandes
print("\nDefiniendo población rural (ID_ENTIDAD == 0 o categorías pequeñas)...")

# Opción más robusta: ID_ENTIDAD == 0 significa área rural/dispersa
df_rural = manzanas_full[manzanas_full['ID_ENTIDAD'] == 0].groupby('COMUNA').agg({
    'TOTAL_PERS': 'sum'
}).reset_index()
df_rural.columns = ['COMUNA', 'POBLACION_RURAL_ID0']

# También contar por AREA_C si existe valor "RURAL"
if 'RURAL' in manzanas_full['AREA_C'].str.upper().values:
    df_rural_area = manzanas_full[manzanas_full['AREA_C'].str.upper().str.contains('RURAL', na=False)].groupby('COMUNA').agg({
        'TOTAL_PERS': 'sum'
    }).reset_index()
    df_rural_area.columns = ['COMUNA', 'POBLACION_RURAL_AREA']
else:
    df_rural_area = pd.DataFrame({'COMUNA': [], 'POBLACION_RURAL_AREA': []})

# 3. Unir ambas métricas
df_comunas_rural = df_comunas_rural.merge(df_rural, on='COMUNA', how='left')
df_comunas_rural = df_comunas_rural.merge(df_rural_area, on='COMUNA', how='left')

df_comunas_rural['POBLACION_RURAL_ID0'] = df_comunas_rural['POBLACION_RURAL_ID0'].fillna(0)
df_comunas_rural['POBLACION_RURAL_AREA'] = df_comunas_rural['POBLACION_RURAL_AREA'].fillna(0)

# Usar el máximo entre ambas definiciones
df_comunas_rural['POBLACION_RURAL'] = df_comunas_rural[['POBLACION_RURAL_ID0', 'POBLACION_RURAL_AREA']].max(axis=1)

# 4. Calcular población urbana (el resto)
df_comunas_rural['POBLACION_URBANA'] = df_comunas_rural['POBLACION_TOTAL'] - df_comunas_rural['POBLACION_RURAL']
df_comunas_rural['POBLACION_URBANA'] = df_comunas_rural['POBLACION_URBANA'].clip(lower=0)

# 5. Calcular porcentaje rural
df_comunas_rural['PCT_RURAL'] = (df_comunas_rural['POBLACION_RURAL'] / df_comunas_rural['POBLACION_TOTAL']).fillna(0)

print(f"Comunas con población rural > 0: {(df_comunas_rural['POBLACION_RURAL'] > 0).sum()}")
print(f"Población rural total: {df_comunas_rural['POBLACION_RURAL'].sum():,.0f}")

# 6. Calcular área total de la comuna (km²)
print("Calculando áreas de comunas...")
comunas_area = manzanas_utm.dissolve(by='COMUNA').reset_index()
comunas_area['AREA_KM2'] = comunas_area.geometry.area / 1_000_000
df_comunas_rural = df_comunas_rural.merge(comunas_area[['COMUNA', 'AREA_KM2']], on='COMUNA', how='left')

# 7. Densidad poblacional
df_comunas_rural['DENSIDAD_HAB_KM2'] = df_comunas_rural['POBLACION_TOTAL'] / df_comunas_rural['AREA_KM2']

# ==========================================
# C. CONTAR PUDOs POR COMUNA
# ==========================================

print("Contando PUDOs existentes por comuna...")

pudos_por_comuna = gdf_PUDO.groupby('COMUNA_NORM').size().reset_index()
pudos_por_comuna.columns = ['COMUNA_NORM', 'PUDOS_EN_COMUNA']

df_comunas_rural['COMUNA_NORM'] = df_comunas_rural['COMUNA'].apply(normalize_text)
df_comunas_rural = df_comunas_rural.merge(pudos_por_comuna, on='COMUNA_NORM', how='left')
df_comunas_rural['PUDOS_EN_COMUNA'] = df_comunas_rural['PUDOS_EN_COMUNA'].fillna(0).astype(int)

# ==========================================
# D. CLASIFICAR OPORTUNIDADES DE HUB RURAL
# ==========================================

print("Clasificando oportunidades de Hub Rural...")

def clasificar_hub_rural(row):
    score = 0
    detalles = []  # Lista de tuplas (factor, puntos, descripción)
    
    # Factor 1: Porcentaje rural (máx 4 pts)
    if row['PCT_RURAL'] >= 0.7:
        pts = 4
        desc = f"Muy alta ruralidad ({row['PCT_RURAL']*100:.0f}%)"
    elif row['PCT_RURAL'] >= 0.5:
        pts = 3
        desc = f"Alta ruralidad ({row['PCT_RURAL']*100:.0f}%)"
    elif row['PCT_RURAL'] >= 0.3:
        pts = 2
        desc = f"Ruralidad media ({row['PCT_RURAL']*100:.0f}%)"
    elif row['PCT_RURAL'] >= 0.1:
        pts = 1
        desc = f"Algo de ruralidad ({row['PCT_RURAL']*100:.0f}%)"
    else:
        pts = 0
        desc = None
    if pts > 0:
        score += pts
        detalles.append(f"📊 Ruralidad: +{pts}pts ({desc})")
    
    # Factor 2: Población rural absoluta (máx 4 pts)
    if row['POBLACION_RURAL'] >= 10000:
        pts = 4
        desc = f"{row['POBLACION_RURAL']:,.0f} hab."
    elif row['POBLACION_RURAL'] >= 5000:
        pts = 3
        desc = f"{row['POBLACION_RURAL']:,.0f} hab."
    elif row['POBLACION_RURAL'] >= 2000:
        pts = 2
        desc = f"{row['POBLACION_RURAL']:,.0f} hab."
    elif row['POBLACION_RURAL'] >= 500:
        pts = 1
        desc = f"{row['POBLACION_RURAL']:,.0f} hab."
    else:
        pts = 0
        desc = None
    if pts > 0:
        score += pts
        detalles.append(f"👥 Pob. Rural: +{pts}pts ({desc})")
    
    # Factor 3: Pocos PUDOs - OPORTUNIDAD (máx 5 pts)
    if row['PUDOS_EN_COMUNA'] == 0:
        pts = 5
        desc = "🚨 GREENFIELD - Sin PUDOs"
    elif row['PUDOS_EN_COMUNA'] == 1:
        pts = 3
        desc = "Solo 1 PUDO existente"
    elif row['PUDOS_EN_COMUNA'] <= 3:
        pts = 1
        desc = f"{row['PUDOS_EN_COMUNA']} PUDOs"
    else:
        pts = 0
        desc = None
    if pts > 0:
        score += pts
        detalles.append(f"📦 PUDOs: +{pts}pts ({desc})")
    
    # Factor 4: Baja densidad (máx 2 pts)
    if row['DENSIDAD_HAB_KM2'] < 10:
        pts = 2
        desc = f"{row['DENSIDAD_HAB_KM2']:.1f} hab/km²"
    elif row['DENSIDAD_HAB_KM2'] < 30:
        pts = 1
        desc = f"{row['DENSIDAD_HAB_KM2']:.1f} hab/km²"
    else:
        pts = 0
        desc = None
    if pts > 0:
        score += pts
        detalles.append(f"📏 Densidad: +{pts}pts ({desc})")
    
    # Factor 5: Área extensa (máx 2 pts)
    if row['AREA_KM2'] > 2000:
        pts = 2
        desc = f"{row['AREA_KM2']:,.0f} km²"
    elif row['AREA_KM2'] > 500:
        pts = 1
        desc = f"{row['AREA_KM2']:,.0f} km²"
    else:
        pts = 0
        desc = None
    if pts > 0:
        score += pts
        detalles.append(f"📐 Área: +{pts}pts ({desc})")
    
    # Factor 6: BONUS - Población total alta en comuna rural (máx 2 pts)
    if row['PCT_RURAL'] >= 0.3 and row['POBLACION_TOTAL'] >= 15000:
        pts = 2
        score += pts
        detalles.append(f"⭐ BONUS: +{pts}pts (Comuna rural grande)")
    elif row['PCT_RURAL'] >= 0.2 and row['POBLACION_TOTAL'] >= 10000:
        pts = 1
        score += pts
        detalles.append(f"⭐ BONUS: +{pts}pts (Comuna con potencial)")
    
    detalle_html = "<br>".join(detalles) if detalles else "Sin factores destacables"
    
    return pd.Series({
        'SCORE_HUB': score,
        'DETALLE_SCORE': detalle_html,
        'RAZONES': ' | '.join([d.split('(')[1].replace(')', '') for d in detalles]) if detalles else 'N/A'
    })

# Aplicar clasificación
scores = df_comunas_rural.apply(clasificar_hub_rural, axis=1)
df_comunas_rural = pd.concat([df_comunas_rural, scores], axis=1)

# Filtrar comunas con potencial
# Score >= 3 Y (tiene población rural O es greenfield)
df_hubs_potenciales = df_comunas_rural[
    (df_comunas_rural['SCORE_HUB'] >= 3) & 
    ((df_comunas_rural['POBLACION_RURAL'] >= 100) | (df_comunas_rural['PUDOS_EN_COMUNA'] == 0))
].copy()
df_hubs_potenciales = df_hubs_potenciales.sort_values('SCORE_HUB', ascending=False)

print(f"Comunas con potencial de Hub Rural: {len(df_hubs_potenciales)}")
print(f"Score máximo: {df_hubs_potenciales['SCORE_HUB'].max()}")
print(f"Score mínimo (filtrado): {df_hubs_potenciales['SCORE_HUB'].min()}")

# ==========================================
# E. CALCULAR CENTROIDES DE COMUNAS
# ==========================================

print("Calculando centroides de comunas...")

comunas_geo = comunas_area.to_crs(epsg=4326)
comunas_geo['centroid'] = comunas_geo.geometry.centroid
comunas_geo['lat'] = comunas_geo['centroid'].y
comunas_geo['lon'] = comunas_geo['centroid'].x

df_hubs_potenciales = df_hubs_potenciales.merge(
    comunas_geo[['COMUNA', 'lat', 'lon']], 
    on='COMUNA', 
    how='left'
)

# ==========================================
# F. CREAR MAPA DE HUBS RURALES
# ==========================================

print("Creando mapa de oportunidades rurales...")

chile_center = [-35.5, -71.5]

mapa_rural = folium.Map(
    location=chile_center,
    zoom_start=5,
    tiles='cartodbpositron'
)

folium.TileLayer('cartodbdark_matter', name='Mapa Oscuro').add_to(mapa_rural)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(mapa_rural)

# ==========================================
# G. CAPA 1: CÍRCULOS DE OPORTUNIDAD
# ==========================================

fg_hubs = folium.FeatureGroup(name='🏔️ Oportunidades Hub Rural', show=True)

# Colores por score - AJUSTADO para mejor distribución
def get_color_by_score(score):
    if score >= 10:
        return 'darkred'    # CRÍTICO
    elif score >= 8:
        return 'red'        # Muy alto
    elif score >= 6:
        return 'orangered'  # Alto
    elif score >= 5:
        return 'orange'     # Medio-Alto
    elif score >= 4:
        return 'gold'       # Medio
    else:
        return 'yellow'     # Base

for idx, row in df_hubs_potenciales.iterrows():
    if pd.notna(row['lat']) and pd.notna(row['lon']):
        # Radio proporcional a población total (más visible)
        pop = max(row['POBLACION_TOTAL'], 500)
        radius = max(8, min(30, np.log10(pop) * 5))
        
        color = get_color_by_score(row['SCORE_HUB'])
        
        # Borde especial si no tiene PUDOs
        if row['PUDOS_EN_COMUNA'] == 0:
            icon_text = "🚨"
            border_color = 'black'
            border_weight = 3
        else:
            icon_text = "🏔️"
            border_color = color
            border_weight = 1
        
        # POPUP CON DETALLE DEL SCORE
        popup_html = f"""
        <div style='width:320px; font-family: Arial, sans-serif;'>
            <h3 style='margin:0 0 5px 0; color:{color}; border-bottom:2px solid {color}; padding-bottom:5px;'>
                {icon_text} {row['COMUNA']}
            </h3>
            
            <div style='background:#f5f5f5; padding:8px; border-radius:5px; margin:5px 0;'>
                <span style='font-size:24px; font-weight:bold; color:{color}'>{row['SCORE_HUB']}</span>
                <span style='font-size:14px; color:#666'> / 19 puntos</span>
            </div>
            
            <table style='width:100%; font-size:12px; border-collapse:collapse;'>
                <tr style='background:#eee;'>
                    <td style='padding:4px;'><b>👥 Pob. Total</b></td>
                    <td style='padding:4px; text-align:right;'>{row['POBLACION_TOTAL']:,.0f}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>🌾 Pob. Rural</b></td>
                    <td style='padding:4px; text-align:right;'><b style='color:green'>{row['POBLACION_RURAL']:,.0f}</b> ({row['PCT_RURAL']*100:.1f}%)</td>
                </tr>
                <tr style='background:#eee;'>
                    <td style='padding:4px;'><b>🏙️ Pob. Urbana</b></td>
                    <td style='padding:4px; text-align:right;'>{row['POBLACION_URBANA']:,.0f}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>📦 PUDOs</b></td>
                    <td style='padding:4px; text-align:right;'><b style='color:{"red" if row["PUDOS_EN_COMUNA"]==0 else "blue"}'>{row['PUDOS_EN_COMUNA']}</b></td>
                </tr>
                <tr style='background:#eee;'>
                    <td style='padding:4px;'><b>📐 Área</b></td>
                    <td style='padding:4px; text-align:right;'>{row['AREA_KM2']:,.0f} km²</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>📏 Densidad</b></td>
                    <td style='padding:4px; text-align:right;'>{row['DENSIDAD_HAB_KM2']:.1f} hab/km²</td>
                </tr>
            </table>
            
            <div style='margin-top:10px; padding:8px; background:#fffde7; border-left:3px solid {color}; font-size:11px;'>
                <b>💡 Desglose del Score:</b><br>
                {row['DETALLE_SCORE']}
            </div>
        </div>
        """
        
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=radius,
            color=border_color,
            weight=border_weight,
            fill=True,
            fillColor=color,
            fillOpacity=0.7,
            popup=folium.Popup(popup_html, max_width=350),
            tooltip=f"{row['COMUNA']} | Score: {row['SCORE_HUB']} | Rural: {row['POBLACION_RURAL']:,.0f} | PUDOs: {row['PUDOS_EN_COMUNA']}"
        ).add_to(fg_hubs)

fg_hubs.add_to(mapa_rural)

# ==========================================
# H. CAPA 2: PUDOs EXISTENTES
# ==========================================

print("Agregando PUDOs existentes...")

pudo_cluster = MarkerCluster(
    name='📍 PUDOs Existentes',
    show=True,
    options={'disableClusteringAtZoom': 10}
)

gdf_pudo_wgs84 = gdf_PUDO.to_crs(epsg=4326)

for idx, row in gdf_pudo_wgs84.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.8,
        popup=f"PUDO: {row['cmns_nmb']}"
    ).add_to(pudo_cluster)

pudo_cluster.add_to(mapa_rural)

# ==========================================
# I. CONTROLES Y LEYENDA
# ==========================================

MiniMap(toggle_display=True).add_to(mapa_rural)
folium.LayerControl(collapsed=False).add_to(mapa_rural)

legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 280px; 
    border:2px solid grey; z-index:9999; 
    background-color:white;
    padding: 10px;
    font-size: 12px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
<b style="font-size:14px">🏔️ Oportunidades Hub Rural</b><br>
<span style="font-size:10px">Comunas donde un PUDO centraliza logística rural</span>
<hr style="margin:5px 0">
<b>Score (máx 19 pts):</b><br>
<span style="color:darkred">●</span> Rojo oscuro = 10+ (CRÍTICO)<br>
<span style="color:red">●</span> Rojo = 8-9 (Muy Alto)<br>
<span style="color:orangered">●</span> Naranja rojo = 6-7 (Alto)<br>
<span style="color:orange">●</span> Naranja = 5 (Medio-Alto)<br>
<span style="color:gold">●</span> Dorado = 4 (Medio)<br>
<span style="color:yellow;background:#333;padding:0 2px">●</span> Amarillo = 3 (Base)<br>
<hr style="margin:5px 0">
<b>Tamaño:</b> Población total<br>
<b>Borde negro:</b> SIN PUDOs 🚨<br>
<span style="color:blue">●</span> = PUDOs existentes
</div>
'''
mapa_rural.get_root().html.add_child(folium.Element(legend_html))

# ==========================================
# J. GUARDAR MAPA Y REPORTE
# ==========================================

filename_mapa_rural = 'Mapa_Oportunidades_Rurales_Hub.html'
mapa_rural.save(filename_mapa_rural)

cols_reporte = [
    'COMUNA', 'SCORE_HUB', 'POBLACION_TOTAL', 'POBLACION_RURAL', 'POBLACION_URBANA',
    'PCT_RURAL', 'PUDOS_EN_COMUNA', 'AREA_KM2', 'DENSIDAD_HAB_KM2', 'RAZONES'
]
df_reporte_hubs = df_hubs_potenciales[cols_reporte].copy()
df_reporte_hubs['PCT_RURAL'] = df_reporte_hubs['PCT_RURAL'].round(3)
df_reporte_hubs['AREA_KM2'] = df_reporte_hubs['AREA_KM2'].round(1)
df_reporte_hubs['DENSIDAD_HAB_KM2'] = df_reporte_hubs['DENSIDAD_HAB_KM2'].round(2)

filename_excel_rural = 'Oportunidades_Hub_Rural.xlsx'
df_reporte_hubs.to_excel(filename_excel_rural, index=False)

filename_all_comunas = 'Analisis_Ruralidad_Comunas.xlsx'
df_comunas_rural.to_excel(filename_all_comunas, index=False)

print(f"\n✅ Mapa generado: {filename_mapa_rural}")
print(f"✅ Top Oportunidades: {filename_excel_rural}")
print(f"✅ Análisis completo: {filename_all_comunas}")

# ==========================================
# K. MOSTRAR RESULTADOS
# ==========================================

print("\n" + "="*75)
print("🏔️ TOP 25 COMUNAS PARA HUB LOGÍSTICO RURAL")
print("="*75)
print(f"{'COMUNA':<22} {'SCORE':>5} {'POB.TOT':>9} {'POB.RUR':>9} {'%RUR':>6} {'PUDOS':>5}")
print("-"*75)

for idx, row in df_reporte_hubs.head(25).iterrows():
    marker = "🚨" if row['PUDOS_EN_COMUNA'] == 0 else "  "
    print(f"{marker}{row['COMUNA']:<20} {row['SCORE_HUB']:>5} {row['POBLACION_TOTAL']:>9,.0f} {row['POBLACION_RURAL']:>9,.0f} {row['PCT_RURAL']*100:>5.0f}% {row['PUDOS_EN_COMUNA']:>5}")

print("\n" + "="*75)
print("🚨 COMUNAS SIN PUDOS CON MAYOR POBLACIÓN (GREENFIELD)")
print("="*75)
greenfield = df_comunas_rural[df_comunas_rural['PUDOS_EN_COMUNA'] == 0].sort_values('POBLACION_TOTAL', ascending=False).head(15)
for idx, row in greenfield.iterrows():
    print(f"  🚨 {row['COMUNA']}: {row['POBLACION_TOTAL']:,.0f} total | {row['POBLACION_RURAL']:,.0f} rural ({row['PCT_RURAL']*100:.0f}%)")

print("\n" + "="*75)
print("📊 ESTADÍSTICAS")
print("="*75)
print(f"Comunas analizadas: {len(df_comunas_rural)}")
print(f"Comunas con potencial Hub: {len(df_hubs_potenciales)}")
print(f"Comunas sin PUDO: {(df_comunas_rural['PUDOS_EN_COMUNA'] == 0).sum()}")
print(f"Población rural total: {df_comunas_rural['POBLACION_RURAL'].sum():,.0f}")


--- Paso 9: Análisis de Oportunidades Rurales (Hub Logístico) ---
Explorando estructura de datos del censo...

Columnas disponibles: ['OBJECTID', 'CUT', 'COD_REGION', 'REGION', 'COD_PROVINCIA', 'PROVINCIA', 'COMUNA', 'AREA_C', 'MANZENT', 'DISTRITO', 'COD_DISTRITO', 'COD_LOCALIDAD', 'COD_ZONA', 'LOCALIDAD', 'COD_ENTIDAD', 'COD_MANZANA', 'ENTIDAD', 'TIPO_MZ', 'COD_CATEGORIA', 'CATEGORIA', 'MZ_BASE_CENSO', 'ID_ENTIDAD', 'ID_LOCALIDAD', 'ID_DISTRITO', 'ID_ZONA', 'TOTAL_PERS', 'n_hombres', 'n_mujeres', 'n_edad_0_5', 'n_edad_6_13', 'n_edad_14_17', 'n_edad_18_24', 'n_edad_25_44', 'n_edad_45_59', 'n_edad_60_mas', 'prom_edad', 'n_inmigrantes', 'n_nacionalidad', 'n_pueblos_orig', 'n_afrodescendencia', 'n_lengua_indigena', 'n_religion', 'n_dificultad_ver', 'n_dificultad_oir', 'n_dificultad_mover', 'n_dificultad_cogni', 'n_dificultad_cuidado', 'n_dificultad_comunic', 'n_discapacidad', 'n_estcivcon_casado', 'n_estcivcon_conviviente', 'n_estcivcon_conv_civil', 'n_estcivcon_anul_sep_div', 'n_estcivco